# Import libraries

In [1]:
import os
import numpy as np

from keras import callbacks
import keras_tuner as kt

from src.data_aug import data_aug_with_random_contrast
from src.ranking_model_prob import create_meta_network

# Prepare data

In [2]:
# Configure question index
question_index = str(1)

# Load training data
training_data_path = 'data/ForModelTraining/train_data/draw_exclude'
img_left_training = np.load(os.path.join(training_data_path + '/train_left_duel_' + question_index + '.npy'), allow_pickle=True)
img_right_training = np.load(os.path.join(training_data_path + '/train_right_duel_' + question_index + '.npy'), allow_pickle=True)
label_training = np.load(os.path.join(training_data_path + '/train_label_duel_' + question_index + '.npy'), allow_pickle=True)

# Load validation data
validation_data_path = 'data/ForModelTraining/validation_data/draw_exclude'
img_left_validation = np.load(os.path.join(validation_data_path + '/val_left_duel_' + question_index + '.npy'), allow_pickle=True)
img_right_validation = np.load(os.path.join(validation_data_path + '/val_right_duel_' + question_index + '.npy'), allow_pickle=True)
label_validation = np.load(os.path.join(validation_data_path + '/val_label_duel_' + question_index + '.npy'), allow_pickle=True)

# Load test data
test_data_path = 'data/ForModelTraining/test_data/draw_exclude'
img_left_test = np.load(os.path.join(test_data_path + '/test_left_duel_' + question_index + '.npy'), allow_pickle=True)
img_right_test = np.load(os.path.join(test_data_path + '/test_right_duel_' + question_index + '.npy'), allow_pickle=True)
label_test = np.load(os.path.join(test_data_path + '/test_label_duel_' + question_index + '.npy'), allow_pickle=True)

In [3]:
print(img_left_training)

[[[[ 59.  71.  71.]
   [ 51.  61.  62.]
   [ 47.  57.  59.]
   ...
   [ 40.  31.  26.]
   [ 44.  35.  30.]
   [ 47.  38.  33.]]

  [[ 65.  76.  80.]
   [ 38.  49.  53.]
   [ 62.  73.  79.]
   ...
   [ 40.  31.  26.]
   [ 42.  33.  28.]
   [ 44.  35.  30.]]

  [[ 91. 101. 111.]
   [ 70.  80.  90.]
   [ 51.  61.  73.]
   ...
   [ 41.  32.  27.]
   [ 42.  33.  28.]
   [ 43.  34.  29.]]

  ...

  [[ 49.  45.  62.]
   [ 51.  47.  64.]
   [ 53.  49.  66.]
   ...
   [ 35.  35.  27.]
   [ 34.  34.  26.]
   [ 34.  34.  26.]]

  [[ 56.  52.  69.]
   [ 57.  53.  70.]
   [ 57.  53.  70.]
   ...
   [ 35.  35.  27.]
   [ 33.  33.  25.]
   [ 32.  32.  24.]]

  [[ 57.  53.  70.]
   [ 53.  49.  66.]
   [ 51.  47.  64.]
   ...
   [ 35.  35.  27.]
   [ 33.  33.  25.]
   [ 31.  31.  23.]]]


 [[[104. 114.  41.]
   [107. 116.  53.]
   [ 88.  94.  48.]
   ...
   [134. 147. 215.]
   [134. 147. 215.]
   [134. 147. 215.]]

  [[ 61.  68.  14.]
   [ 72.  79.  28.]
   [ 77.  82.  42.]
   ...
   [134. 147. 215.]
 

## Augment data with random contrast
*This is a data augmentation method that raised by Hannick. It helps to improve the model performance.*
[Hannick's GitHub](https://github.com/Hannick5/UrbanPerception)

In [4]:
img_left_training,img_right_training,label_training = data_aug_with_random_contrast(img_left_training,img_right_training, label_training,save_folder=None)

In [5]:

label_training = np.argmax(label_training, axis=1)
label_validation = np.argmax(label_validation, axis=1)
label_test = np.argmax(label_test, axis=1)

# Define hptuning model

In [6]:
# Define model for hyperparameters tuning
def build_hptuning_ranking_model(hp):
    # Define model architecture
    model = create_meta_network(
        dense_units=hp.Choice('dense_units', values=[32, 64, 128, 256, 512]),
        dropout_rate=hp.Float('dropout_rate', min_value=0.0, max_value=0.6, step=0.2),
        learning_rate=hp.Choice('learning_rate', values=[1e-4, 1e-5, 1e-6]),
        optimizer=hp.Choice('optimizer', values=['adam', 'rmsprop', 'sgd']),
        activation=hp.Choice('activation', values=['relu', 'tanh', 'sigmoid']),
        learning_rate_decay=hp.Choice('learning_rate_decay', values=[1e-3, 1e-4, 1e-5]),
        img_size=int(224),
        data_aug=True,
        weights=None
    )
    return model
build_hptuning_ranking_model(kt.HyperParameters())
# Adopt Bayesian optimization
tuner = kt.BayesianOptimization(
    build_hptuning_ranking_model,
    objective='val_accuracy',
    max_trials=20,
    executions_per_trial=4,
    overwrite=False,
    directory='model_results/Q'+ question_index +'/hptuning_ranking_model',
    project_name="ranking_model_prob_230723",
)

# Tune hyperparameters

In [ ]:
# Employ early stopping to prevent overfitting
early_stopping = callbacks.EarlyStopping(
    monitor="val_loss",
    patience=10,
    mode='min',
    restore_best_weights=True
)

# Tune hyperparameters
tuner.search(
    x=[img_left_training, img_right_training],
    y=label_training,
    epochs=50,
    batch_size=32,
    validation_data=([img_left_validation, img_right_validation], label_validation),
    callbacks=[early_stopping]
)


Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
64                |64                |dense_units
0.4               |0.4               |dropout_rate
1e-06             |1e-06             |learning_rate
rmsprop           |rmsprop           |optimizer
relu              |relu              |activation
0.001             |0.001             |learning_rate_decay

Epoch 1/50
172/172 [==============================] - 25s 118ms/step - loss: 1.1195 - accuracy: 0.5272 - val_loss: 0.7592 - val_accuracy: 0.6024
Epoch 2/50
172/172 [==============================] - 18s 104ms/step - loss: 1.0181 - accuracy: 0.5599 - val_loss: 0.6940 - val_accuracy: 0.6353
Epoch 3/50
172/172 [==============================] - 18s 102ms/step - loss: 0.9296 - accuracy: 0.5908 - val_loss: 0.6547 - val_accuracy: 0.6681
Epoch 4/50
172/172 [==============================] - 18s 104ms/step - loss: 0.8861 - accuracy: 0.6217 - val_loss: 0.6264 - val_accuracy: 0.6867
Epoch 5/50
172/172 [===========

# Obtain hptuning results

In [ ]:
# Display the tuning process summary
tuner.results_summary()

In [ ]:
# Save the best five hptuned model architectures
best_hps = tuner.get_best_hyperparameters(5)
save_dir = "model_results/Q2/hp_tuning/hptuning_ranking_230521"
for i in range(len(best_hps)):
    model = build_hptuning_ranking_model(best_hps[i])
    model.save(os.path.join(save_dir, 'best_model_' + str(i) + '_architecture.h5'))